In [5]:
import pandas as pd, pathlib
root = pathlib.Path("/Users/atharvramesh/UCSD/Fall2025/ECE225A/NYC_Public_Transit/data/raw/citibike")
files = sorted(root.glob("202401-citibike-tripdata_*.csv"))
df = pd.concat(pd.read_csv(f) for f in files)

/var/folders/hw/zpghgr8x4ybc9d8tnpvwmn780000gn/T/ipykernel_99003/3073021474.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(pd.read_csv(f) for f in files)
/var/folders/hw/zpghgr8x4ybc9d8tnpvwmn780000gn/T/ipykernel_99003/3073021474.py:4: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(pd.read_csv(f) for f in files)


In [37]:
root = pathlib.Path("/Users/atharvramesh/UCSD/Fall2025/ECE225A/NYC_Public_Transit/data/raw/citibike")
files = sorted(root.glob("202401-citibike-tripdata_*.csv"))
pd.read_csv(files[0])

/var/folders/hw/zpghgr8x4ybc9d8tnpvwmn780000gn/T/ipykernel_99003/407845358.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(files[0])


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,8E865410DBDE0CA9,electric_bike,2024-01-01 13:00:04.563,2024-01-01 13:04:04.652,3 St & 3 Ave,4028.03,Carroll St & Smith St,4225.14,40.675070,-73.987752,40.680611,-73.994758,casual
1,0403D0B3FC9CA77D,electric_bike,2024-01-08 19:36:43.520,2024-01-08 19:53:16.266,Franklin Ave & St Marks Ave,4107.05,Bedford Ave & Bergen St,4066.15,40.675832,-73.956168,40.676368,-73.952918,casual
2,F6DE7BB42FF550BE,electric_bike,2024-01-12 15:00:41.580,2024-01-12 15:36:29.622,W 67 St & Broadway,7116.04,Central Park W & W 103 St,7577.27,40.774925,-73.982666,40.795590,-73.961884,casual
3,84A995BFD98030D4,classic_bike,2024-01-12 16:52:19.025,2024-01-12 17:17:29.773,Central Park West & W 68 St,7079.06,E 5 St & Ave C,5545.04,40.773407,-73.977825,40.722992,-73.979955,member
4,7BBEAD4F2B535813,electric_bike,2024-01-05 19:50:19.202,2024-01-05 20:34:42.517,W 67 St & Broadway,7116.04,Ave A & E 14 St,5779.11,40.774925,-73.982666,40.730311,-73.980472,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,53C5A7DDC930FE68,electric_bike,2024-01-19 08:11:31.812,2024-01-19 08:18:23.659,Devoe St & Lorimer St,5259.06,Meserole Ave & Manhattan Ave,5666.04,40.713352,-73.949103,40.727086,-73.952991,member
999996,80C14103C204CD47,electric_bike,2024-01-23 08:50:57.782,2024-01-23 09:12:37.637,Fulton St & Waverly Ave,4345.11,Broad St & Bridge St,4962.08,40.683239,-73.965996,40.703652,-74.011678,member
999997,E1EE9037A8CB6101,electric_bike,2024-01-21 14:16:24.984,2024-01-21 14:22:14.498,Driggs Ave & N 9 St,5411.08,Meserole Ave & Manhattan Ave,5666.04,40.718170,-73.955201,40.727086,-73.952991,member
999998,DA5E681B117190A0,electric_bike,2024-01-23 09:02:49.848,2024-01-23 09:21:19.513,Driggs Ave & N 9 St,5411.08,Pearl St & Hanover Square,4993.02,40.718170,-73.955201,40.704718,-74.009260,member


In [14]:
trips = df

In [15]:
trips["started_at"] = pd.to_datetime(trips["started_at"], utc=False)
trips["ended_at"] = pd.to_datetime(trips["ended_at"], utc=False)
trips["trip_minutes"] = (trips["ended_at"] - trips["started_at"]).dt.total_seconds() / 60
mask = (trips["trip_minutes"] > 0) & (trips["trip_minutes"] < 240)
trips = trips[mask & trips["start_station_id"].notna()].copy()
trips["start_station_id"] = trips["start_station_id"].astype(str)

In [18]:
grouped = (
        trips.groupby([
            trips["start_station_id"],
            trips["started_at"].dt.floor("15min"),
        ])
        .size()
        .rename("rides")
        .reset_index()
    )
counts = grouped.pivot_table(
    index="started_at", columns="start_station_id", values="rides", fill_value=0
)

In [19]:
counts

start_station_id,2733.03,2782.02,2821.05,2832.03,2861.02,2872.02,2883.03,2898.01,2912.08,2923.01,...,8866.01,8876.04,8879.02,8879.03,8897.05,JC009,JC023,SYS016,SYS033,SYS038
started_at,,,,,,,,,,,,,,,,,,,,,
2023-12-31 21:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-12-31 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-12-31 22:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-12-31 22:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-12-31 22:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-31 22:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-01-31 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-01-31 23:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
station_totals = counts.sum().sort_values(ascending=False)
print(station_totals)

start_station_id
6140.05           8325.0
6822.09           6568.0
6450.05           6343.0
6331.01           6027.0
6197.08           5903.0
                   ...  
6173.08_Pillar       2.0
JC009                1.0
JC023                1.0
6780.06              1.0
SYS038               1.0
Length: 2223, dtype: float64


In [23]:
top = station_totals.head(100).index

In [24]:
top

Index(['6140.05', '6822.09', '6450.05', '6331.01', '6197.08', '5779.11',
       '6173.08', '5788.13', '6602.03', '5905.12', '5905.14', '5980.07',
       '6306.01', '6364.07', '5303.06', '6072.11', '7260.09', '6257.03',
       '6756.01', '6726.01', '6526.05', '6756.05', '6306.06', '6303.01',
       '6753.08', '6551.11', '6459.07', '5374.01', '5329.03', '7175.05',
       '5886.13', '5382.07', '6247.06', '5989.02', '6464.09', '5382.06',
       '6912.01', '7059.08', '6389.09', '6432.11', '5308.04', '5626.07',
       '6122.09', '6584.12', '6072.06', '5303.08', '5854.09', '6889.12',
       '7212.05', '6289.06', '5712.03', '6280.12', '7020.02', '7014.12',
       '5763.03', '5561.06', '5593.01', '6925.09', '7116.04', '6459.04',
       '6004.06', '6955.05', '5636.11', '6182.02', '5492.05', '7207.08',
       '5929.01', '6266.06', '6762.02', '6535.04', '6611.02', '6560.01',
       '6022.04', '6747.07', '6953.08', '6676.02', '5679.08', '7311.02',
       '6098.12', '5340.01', '6064.08', '5470.12', 

In [28]:
from scipy.stats import chisquare
from scipy.stats import nbinom as sp_nbinom
from scipy.stats import poisson as sp_poisson
from typing import Dict, Tuple
import numpy as np
from pathlib import Path
from plotly import graph_objects as go
from plotly import io as pio


def poisson_summary(series: pd.Series) -> Dict[str, float]:
    lam = series.mean()
    variance = series.var(ddof=0)
    dispersion = variance / lam if lam > 0 else np.nan
    value_counts = series.value_counts()
    xs = value_counts.index.values
    obs = value_counts.values
    expected = sp_poisson.pmf(xs, lam) * len(series)
    mask = expected > 5
    if mask.any():
        obs_masked = obs[mask]
        exp_masked = expected[mask]
        scale = obs_masked.sum() / exp_masked.sum()
        exp_masked *= scale
        chi2_stat, chi2_p = chisquare(obs_masked, exp_masked)
    else:
        chi2_stat = np.nan
        chi2_p = np.nan
    return {
        "mean": float(lam),
        "variance": float(variance),
        "dispersion_index": float(dispersion),
        "chi2_stat": float(chi2_stat),
        "chi2_pvalue": float(chi2_p),
    }


def estimate_nb_params(series: pd.Series) -> Tuple[float, float]:
    mean = series.mean()
    var = series.var(ddof=0)
    if var <= mean or mean <= 0:
        return np.nan, np.nan
    r = mean ** 2 / (var - mean)
    p = r / (r + mean)
    return float(r), float(p)


def make_histogram(
    series: pd.Series, station: str, output_dir: Path, freq: str, nb_params: Tuple[float, float]
) -> None:
    lam = series.mean()
    grid = np.arange(0, max(series.max(), int(series.quantile(0.99)) + 5) + 1)
    observed = series.value_counts().reindex(grid, fill_value=0).values
    fig = go.Figure()
    fig.add_bar(x=grid, y=observed, name="Observed", marker=dict(color="#4B6BFB"), opacity=0.75)
    poisson_expected = sp_poisson.pmf(grid, lam) * len(series)
    fig.add_scatter(
        x=grid, y=poisson_expected, mode="lines", name="Poisson", line=dict(color="#FFA500", width=2)
    )
    nb_r, nb_p = nb_params
    if np.isfinite(nb_r) and np.isfinite(nb_p) and nb_r > 0 and 0 < nb_p < 1:
        nb_expected = sp_nbinom.pmf(grid, nb_r, nb_p) * len(series)
        fig.add_scatter(
            x=grid,
            y=nb_expected,
            mode="lines",
            name="Neg-Bin",
            line=dict(color="#D62728", width=3),
        )
    fig.update_layout(
        title=f"Station {station} ({freq})",
        xaxis_title="Arrivals per bucket",
        yaxis_title="Frequency",
        template="plotly_white",
    )
    output_dir.mkdir(parents=True, exist_ok=True)
    base = station.replace("/", "_").replace(" ", "_")
    html = output_dir / f"{base}.html"
    pdf = output_dir / f"{base}.pdf"
    fig.write_html(html)
    try:
        pio.write_image(fig, pdf)
    except Exception as exc:  # noqa: BLE001
        print(f"Warning: could not export histogram for {station}: {exc}")

In [34]:
summaries = {}
# hist_dir = args.output / "histograms"
for station in top:
    series = counts[station]
    stats = poisson_summary(series)
    nb_r, nb_p = estimate_nb_params(series)
    # print(f"Station {station}: Poisson mean={stats['mean']}, Neg-Bin r={nb_r}, p={nb_p}")
    # stats["nb_r"] = nb_r
    nb_mean = float(nb_r * (1 - nb_p) / nb_p)
    nb_var = float(nb_r * (1 - nb_p) / (nb_p ** 2))
    print(f"Station {station}: Neg-Bin mean={nb_mean}, variance={nb_var}")
    print(f"Station {station}: Mean = {stats['mean']}, variance={stats['variance']}")
    # stats["nb_p"] = nb_p
    # summaries[station] = stats
    # make_histogram(series, station, hist_dir, args.freq, (nb_r, nb_p))

Station 6140.05: Neg-Bin mean=2.788944723618091, variance=10.512239589909349
Station 6140.05: Mean = 2.7889447236180906, variance=10.512239589909345
Station 6822.09: Neg-Bin mean=2.2003350083752093, variance=7.780971412057497
Station 6822.09: Mean = 2.2003350083752093, variance=7.780971412057496
Station 6450.05: Neg-Bin mean=2.1249581239530984, variance=7.022576422031989
Station 6450.05: Mean = 2.124958123953099, variance=7.0225764220319915
Station 6331.01: Neg-Bin mean=2.019095477386934, variance=6.112533185188925
Station 6331.01: Mean = 2.0190954773869345, variance=6.112533185188926
Station 6197.08: Neg-Bin mean=1.9775544388609718, variance=5.519429195110112
Station 6197.08: Mean = 1.9775544388609716, variance=5.519429195110112
Station 5779.11: Neg-Bin mean=1.970184254606365, variance=4.460417553989938
Station 5779.11: Mean = 1.9701842546063653, variance=4.460417553989939
Station 6173.08: Neg-Bin mean=1.9480737018425462, variance=6.581223257549616
Station 6173.08: Mean = 1.9480737018